# Demand prediction baseline solution

Victor Kantor, xead.wl@gmail.com

In [18]:
import pandas as pd
import numpy as np

train = pd.read_csv("train.tsv")
test = pd.read_csv("test.tsv")
sample_submission = pd.read_csv("sample_submission.tsv")

In [19]:
from sklearn.model_selection import train_test_split

frac = 1 # fraction of learning examples used for model fitting

# sample the train set if your don't want to deel with all examples
train = train.sample(frac=frac, random_state=42)

X = train.drop(['Num','y'], axis=1)
y = train['y']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=1)

In [20]:
important_features = ['week', 'item_id', 'shift', 'year', 'f1', 'f2', 'f3', 'f4', 'f5']

In [21]:
X_test = test[important_features]

In [22]:
#xgbr = xgb.XGBRegressor(max_depth=15, n_estimators=1000, gamma=1)
# xgbr.fit(X[important_features], y)
# preds = xgbr.predict()

In [23]:
# new_predicitons = xgbr.predict(X_test)

In [24]:
def loss_func(F, A):
    if len(F) != len(A):
        raise BaseException
    res = 0
    for i in range(len(F)):
        res += float(abs(F[i] - A[i]) / (abs(A[i]) + abs(F[i])))
    res *= float(200) / len(F)
    return res

In [25]:
from sklearn.metrics import make_scorer
scorer = make_scorer(loss_func)

In [26]:
# XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=1, gamma=1,
#        learning_rate=0.1, max_delta_step=0, max_depth=15,
#        min_child_weight=1, missing=None, n_estimators=1000, nthread=-1,
#        objective='reg:linear', reg_alpha=0, reg_lambda=1,
#        scale_pos_weight=1, seed=0, silent=True, subsample=1)

In [32]:
import xgboost as xgb

clf_config = {
    #'GradientBoostingRegressor' : (GradientBoostingRegressor(), {'n_estimators' : range(300,301), 'max_depth' : range(3,8)})
    'XGBRegressor' : (xgb.XGBRegressor(), {'max_depth' : range(15,21), 
                                           'n_estimators' : range(90,126,5),
                                           'learning_rate' : np.linspace(0.01, 0.3, 5),
                                           'min_child_weight' : range(0,5),
                                           'reg_alpha' : range(0,3),
#                                            'reg_labmda' : range(0,3),
                                           'scale_pos_weight' : range(0,3),})
}

In [ ]:
from sklearn.model_selection import GridSearchCV

cv_values = {}
for name, (clf, args) in clf_config.items():
    print('Tuning parameters for %s' % name)
    gs_clf = GridSearchCV(estimator=clf, param_grid=args, scoring=scorer)
    gs_clf.fit(np.array(X[important_features]), np.array(y))
    
    print("\tBest params: {}".format(gs_clf.cv_results_))

Tuning parameters for XGBRegressor


In [ ]:
#model = #GradientBoostingRegressor(n_estimators=100, max_depth=5, random_state=43)
# model = xgb.XGBRegressor(n_estimators=400, max_depth=10)
# model.fit(X_train, y_train)


In [104]:
preds = model.predict(np.array(test.drop(['Num'], axis=1)))

In [105]:
import numpy as np
preds = np.array(preds)
y_test = np.array(y_test)

In [106]:
loss_func(preds, y_test)

BaseException: 

In [183]:
sample_submission['y'] = new_predicitons

In [184]:
sample_submission.head(5)

,Num,y
0,348622,1467.554199
1,348623,16877.605469
2,348624,212119.640625
3,348625,30401.132812
4,348626,3810.147461


In [185]:
# In GBM you can get some negative predictions:
print sample_submission[sample_submission['y'] < 0]

         Num          y
399   349692 -49.195480
625   349918 -82.665092
851   350364 -49.195480
1077  350590 -49.195480
1078  350591 -51.080593
1523  351036 -40.327545
1524  351037 -51.080593


In [186]:
sample_submission['y'] = sample_submission['y'].map(lambda x: x if x > 0 else 0.0)

In [187]:
sample_submission.to_csv("baseline_submission.tsv", sep=',', index=False)

In [ ]:
import numpy as np
range(100, 501, 40)

NameError: name 'linspace' is not defined